In [5]:
!pip install gensim tensorflow scikit-learn matplotlib seaborn

In [95]:
import numpy as np
import pandas as pd
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam

In [96]:
df = pd.read_csv('merged_dataset_new.csv')

In [1]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [3]:
!pip install gensim tensorflow scikit-learn matplotlib seaborn

In [5]:
import gensim
pretrained_model_path = '/content/drive/My Drive/BioWordVec/bio_embedding_extrinsic'
pretrained_model = gensim.models.KeyedVectors.load_word2vec_format(pretrained_model_path, binary=True)

vocab_size = len(pretrained_model)
print(f"Pre-trained model vocabulary size: {vocab_size}")

Pre-trained model vocabulary size: 2324849


In [97]:
vector_size = pretrained_model.vector_size
print(f"Vector size of the pre-trained model: {vector_size}")


Vector size of the pre-trained model: 200


In [98]:
def preprocess_text(text):
    tokens = text.split()
    tokens = [word for word in tokens if word in pretrained_model.key_to_index]
    return tokens

def text_to_vector(text):
    tokens = preprocess_text(text)
    if not tokens:
        return np.zeros(200)
    return np.mean([pretrained_model[word] for word in tokens], axis=0)


In [99]:
df['Type_vector'] = df['Type'].apply(lambda x: text_to_vector(x))
df['Statement_vector'] = df['Statement'].apply(lambda x: text_to_vector(x))
df['Reference_vector'] = df['Reference_data'].apply(lambda x: text_to_vector(x))

In [100]:
df.head()

,test_ID,Type,Section_id,Primary_id,Secondary_id,Statement,Label,Primary_evidence_index,Secondary_evidence_index,Reference_data,Type_vector,Statement_vector,Reference_vector
0,5bc844fc-e852-4270-bfaf-36ea9eface3d,Comparison,Intervention,NCT01928186,NCT00684983,All the primary trial participants do not rece...,Contradiction,INTERVENTION 1: Diagnostic (FLT PET) Pati...,INTERVENTION 1: Arm A Patients receive or...,INTERVENTION 1: Diagnostic (FLT PET) Pati...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.20870005, 0.010578745, -0.14042668, -0.1325...","[0.27742657, -0.0015234367, -0.14305797, -0.01..."
1,86b7cb3d-6186-4a04-9aa6-b174ab764eed,Single,Eligibility,NCT00662129,NaN,"Patients with Platelet count over 100,000/mm¬¨...",Contradiction,"PATIENT CHARACTERISTICS: ANC 1,500/mm³ ...",NaN,DISEASE CHARACTERISTICS: Histologically or c...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.27734584, 0.14244205, -0.083380654, 0.02588...","[0.24579634, 0.06359309, -0.19657847, -0.04272..."
2,dbed5471-c2fc-45b5-b26f-430c9fa37a37,Comparison,Adverse Events,NCT00093145,NCT00703326,Heart-related adverse events were recorded in ...,Entailment,Adverse Events 1: Supraventricular tachycard...,Adverse Events 1: Atrial fibrillation 1/752 ...,Adverse Events 1: Total: 5/32 (15.63%) Feb...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.26496604, 0.15919074, -0.11285321, -0.18515...","[0.15163456, 0.20475844, -0.24133596, 0.050696..."
3,20c35c89-8d23-4be3-b603-ac0ee0f3b4de,Single,Eligibility,NCT01097642,NaN,Adult Patients with histologic confirmation of...,Contradiction,Inclusion Criteria: Patients with histologic...,NaN,Inclusion Criteria: Patients with histologic...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.13775751, 0.020728718, -0.15459715, -0.1009...","[0.21726197, 0.1062329, -0.1627092, -0.0487415..."
4,f17cb242-419d-4f5d-bfa4-41494ed5ac0e,Comparison,Intervention,NCT00852930,NCT02308020,Laser Therapy is in each cohort of the primary...,Contradiction,INTERVENTION 1: Laser Therapy Alone thera...,"INTERVENTION 1: Part A Abemaciclib: HR+, HE...",INTERVENTION 1: Laser Therapy Alone thera...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.2073716, 0.14273366, -0.12802854, -0.110927...","[0.21484275, -0.03353516, -0.19706348, -0.0449..."


In [101]:
X = np.hstack([df['Type_vector'].values.tolist(),
               df['Statement_vector'].values.tolist(),
               df['Reference_vector'].values.tolist()])

In [102]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Label'])

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [104]:
kf = StratifiedKFold(n_splits=5)
accuracies = []
precisions = []
recalls = []
f1s = []

for train_index, val_index in kf.split(X, y):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    model = Sequential()
    model.add(Dense(512, input_dim=X_train_scaled.shape[1], activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint('Label_Prediction.h5', save_best_only=True, monitor='val_loss', mode='min')

    history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_val_scaled, y_val), callbacks=[early_stopping, model_checkpoint])

    model.load_weights('Label_Prediction.h5')
    y_val_pred = (model.predict(X_val_scaled) > 0.5).astype("int32")

    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    f1 = f1_score(y_val, y_val_pred)

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)

print(f'Cross-validated accuracy: {np.mean(accuracies)}')
print(f'Cross-validated precision: {np.mean(precisions)}')
print(f'Cross-validated recall: {np.mean(recalls)}')
print(f'Cross-validated F1 score: {np.mean(f1s)}')

Epoch 1/100
43/43 [==============================] - 3s 13ms/step - loss: 0.9657 - accuracy: 0.5147 - val_loss: 0.7173 - val_accuracy: 0.4912
Epoch 2/100
20/43 [============>.................] - ETA: 0s - loss: 0.8903 - accuracy: 0.5266

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


43/43 [==============================] - 0s 7ms/step - loss: 0.9345 - accuracy: 0.5029 - val_loss: 0.7373 - val_accuracy: 0.4941
Epoch 3/100
43/43 [==============================] - 0s 7ms/step - loss: 0.8855 - accuracy: 0.5000 - val_loss: 0.7533 - val_accuracy: 0.4853
Epoch 4/100
43/43 [==============================] - 0s 7ms/step - loss: 0.8853 - accuracy: 0.4838 - val_loss: 0.7585 - val_accuracy: 0.4824
Epoch 5/100
43/43 [==============================] - 0s 7ms/step - loss: 0.8403 - accuracy: 0.4919 - val_loss: 0.7628 - val_accuracy: 0.4706
Epoch 6/100
43/43 [==============================] - 0s 7ms/step - loss: 0.8307 - accuracy: 0.5000 - val_loss: 0.7661 - val_accuracy: 0.4559
Epoch 7/100
43/43 [==============================] - 0s 6ms/step - loss: 0.7844 - accuracy: 0.5257 - val_loss: 0.7634 - val_accuracy: 0.4676
Epoch 8/100
43/43 [==============================] - 0s 6ms/step - loss: 0.7759 - accuracy: 0.5213 - val_loss: 0.7655 - val_accuracy: 0.4559
Epoch 9/100
43/43 [======

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


43/43 [==============================] - 0s 6ms/step - loss: 0.8169 - accuracy: 0.5250 - val_loss: 0.7076 - val_accuracy: 0.4765
Epoch 3/100
43/43 [==============================] - 0s 6ms/step - loss: 0.8210 - accuracy: 0.5066 - val_loss: 0.7166 - val_accuracy: 0.4647
Epoch 4/100
43/43 [==============================] - 0s 6ms/step - loss: 0.8083 - accuracy: 0.5044 - val_loss: 0.7261 - val_accuracy: 0.4500
Epoch 5/100
43/43 [==============================] - 0s 7ms/step - loss: 0.7917 - accuracy: 0.5191 - val_loss: 0.7333 - val_accuracy: 0.4441
Epoch 6/100
43/43 [==============================] - 0s 7ms/step - loss: 0.7695 - accuracy: 0.5250 - val_loss: 0.7414 - val_accuracy: 0.4353
Epoch 7/100
43/43 [==============================] - 0s 6ms/step - loss: 0.7716 - accuracy: 0.5221 - val_loss: 0.7470 - val_accuracy: 0.4412
Epoch 8/100
43/43 [==============================] - 0s 6ms/step - loss: 0.7640 - accuracy: 0.5287 - val_loss: 0.7483 - val_accuracy: 0.4294
Epoch 9/100
43/43 [======

...

43/43 [==============================] - 0s 6ms/step - loss: 0.8987 - accuracy: 0.5132 - val_loss: 0.7129 - val_accuracy: 0.4853
Epoch 3/100
43/43 [==============================] - 0s 6ms/step - loss: 0.8538 - accuracy: 0.5066 - val_loss: 0.7168 - val_accuracy: 0.4647
Epoch 4/100
43/43 [==============================] - 0s 7ms/step - loss: 0.8610 - accuracy: 0.4860 - val_loss: 0.7224 - val_accuracy: 0.4824
Epoch 5/100
43/43 [==============================] - 0s 6ms/step - loss: 0.8198 - accuracy: 0.4912 - val_loss: 0.7288 - val_accuracy: 0.4882
Epoch 6/100
43/43 [==============================] - 0s 6ms/step - loss: 0.7970 - accuracy: 0.5103 - val_loss: 0.7341 - val_accuracy: 0.4882
Epoch 7/100
43/43 [==============================] - 0s 6ms/step - loss: 0.7945 - accuracy: 0.5051 - val_loss: 0.7382 - val_accuracy: 0.4676
Epoch 8/100
43/43 [==============================] - 0s 7ms/step - loss: 0.7778 - accuracy: 0.5051 - val_loss: 0.7387 - val_accuracy: 0.4647
Epoch 9/100
43/43 [======

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


43/43 [==============================] - 0s 7ms/step - loss: 0.8172 - accuracy: 0.4941 - val_loss: 0.7127 - val_accuracy: 0.4500
Epoch 3/100
43/43 [==============================] - 0s 7ms/step - loss: 0.8108 - accuracy: 0.4985 - val_loss: 0.7173 - val_accuracy: 0.4824
Epoch 4/100
43/43 [==============================] - 0s 6ms/step - loss: 0.8119 - accuracy: 0.4875 - val_loss: 0.7217 - val_accuracy: 0.4853
Epoch 5/100
43/43 [==============================] - 0s 7ms/step - loss: 0.8242 - accuracy: 0.4816 - val_loss: 0.7286 - val_accuracy: 0.4500
Epoch 6/100
43/43 [==============================] - 0s 7ms/step - loss: 0.7510 - accuracy: 0.5390 - val_loss: 0.7352 - val_accuracy: 0.4294
Epoch 7/100
43/43 [==============================] - 0s 7ms/step - loss: 0.7745 - accuracy: 0.5103 - val_loss: 0.7397 - val_accuracy: 0.4235
Epoch 8/100
43/43 [==============================] - 0s 7ms/step - loss: 0.7530 - accuracy: 0.5147 - val_loss: 0.7461 - val_accuracy: 0.4147
Epoch 9/100
43/43 [======

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


43/43 [==============================] - 0s 6ms/step - loss: 0.7659 - accuracy: 0.5301 - val_loss: 0.7159 - val_accuracy: 0.4882
Epoch 3/100
43/43 [==============================] - 0s 7ms/step - loss: 0.7649 - accuracy: 0.5228 - val_loss: 0.7248 - val_accuracy: 0.4853
Epoch 4/100
43/43 [==============================] - 0s 7ms/step - loss: 0.7583 - accuracy: 0.5243 - val_loss: 0.7353 - val_accuracy: 0.4647
Epoch 5/100
43/43 [==============================] - 0s 7ms/step - loss: 0.7667 - accuracy: 0.5169 - val_loss: 0.7407 - val_accuracy: 0.4441
Epoch 6/100
43/43 [==============================] - 0s 6ms/step - loss: 0.7399 - accuracy: 0.5169 - val_loss: 0.7474 - val_accuracy: 0.4441
Epoch 7/100
43/43 [==============================] - 0s 6ms/step - loss: 0.7554 - accuracy: 0.5081 - val_loss: 0.7521 - val_accuracy: 0.4324
Epoch 8/100
43/43 [==============================] - 0s 7ms/step - loss: 0.7580 - accuracy: 0.5015 - val_loss: 0.7572 - val_accuracy: 0.4294
Epoch 9/100
43/43 [======

In [106]:
model.load_weights('Label_Prediction.h5')

y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'Confusion matrix:\n {conf_matrix}')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

11/11 [==============================] - 0s 2ms/step
Confusion matrix:
 [[  8 164]
 [ 23 145]]
Accuracy: 0.45
Precision: 0.4692556634304207
Recall: 0.8630952380952381
F1 Score: 0.6079664570230607


In [ ]:
coling

In [ ]:
aaai